In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as split
from sklearn.utils import shuffle

learning_rate = 0.005
batch_size = 100
epoch = 10000
log_interval = 1000

In [2]:
# Features Selection
dataset = shuffle(pd.read_csv('student-mat.csv'))
Y = dataset[['romantic']]
X = dataset[['sex', 'age', 'famsize', 'traveltime', 'studytime', 'activities', 'goout', 'famrel']]
Y['romantic'][Y['romantic']=='no'] = 0
Y['romantic'][Y['romantic']=='yes'] = 1
X['sex'][X['sex']=='F'] = 0
X['sex'][X['sex']=='M'] = 1
X['famsize'][X['famsize']=='LE3'] = 0
X['famsize'][X['famsize']=='GT3'] = 1
X['activities'][X['activities']=='no'] = 0
X['activities'][X['activities']=='yes'] = 1
X = X.apply(pd.to_numeric)
# Splitting into train set, validation set and test set
def train_val_test_split(X, Y, dist):
    X, Y = X.as_matrix().astype(float), Y.as_matrix().astype(float)
    a, b, c = np.array(dist)
    train_x, tmp_x, train_y, tmp_y = split(X, Y, test_size=(b+c)/(a+b+c))
    val_x, test_x, val_y, test_y = split(tmp_x, tmp_y, test_size=c/(b+c))
    return train_x, val_x, test_x, train_y, val_y, test_y
train_x, val_x, test_x, train_y, val_y, test_y = train_val_test_split(X, Y, [7, 1, 2])
m_train = train_x.shape[0]
X.describe()

/home/wermarter/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/wermarter/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/wermarter/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/wermarter/an

,sex,age,famsize,traveltime,studytime,activities,goout,famrel
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.473418,16.696203,0.711392,1.448101,2.035443,0.508861,3.108861,3.944304
std,0.499926,1.276043,0.453690,0.697505,0.839240,0.500555,1.113278,0.896659
min,0.000000,15.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,0.000000,16.000000,0.000000,1.000000,1.000000,0.000000,2.000000,4.000000
50%,0.000000,17.000000,1.000000,1.000000,2.000000,1.000000,3.000000,4.000000
75%,1.000000,18.000000,1.000000,2.000000,2.000000,1.000000,4.000000,5.000000
max,1.000000,22.000000,1.000000,4.000000,4.000000,1.000000,5.000000,5.000000


In [3]:
# Data for prediction
sex, age, famsize, traveltime, studytime, activities, goout, famrel = 1, 17, 1, 1, 5, 0, 1, 4
input_data = np.array([sex, age, famsize, traveltime, studytime, activities, goout, famrel]).astype(float)

In [4]:
# Model
def sigm_activation(X):
    shape = tf.shape(X)
    condition = tf.less(X, tf.fill(shape, 0.5))
    return tf.where(condition, tf.zeros(shape), tf.ones(shape))
ndim = len(input_data)

X = tf.placeholder(tf.float32, [None, ndim])
y = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(np.random.randn(ndim, 1).astype('float32'))
b = tf.Variable(np.random.randn(1).astype('float32'))
z = tf.add(tf.matmul(X, W), b)

_pred = tf.sigmoid(z)
#L = -tf.reduce_mean((1-y)*tf.log(1-_pred)+y*tf.log(_pred))
L = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)
L = tf.reduce_mean(L)

optim = tf.train.AdamOptimizer(learning_rate).minimize(L)
_pred_thresholded = sigm_activation(_pred)
_correct = tf.equal(y, _pred_thresholded)
acc = tf.reduce_mean(tf.cast(_correct, tf.float32))

In [5]:
# Train it
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for _ in range(int(epoch*(m_train/batch_size))):
    randindx = np.random.randint(m_train, size=batch_size)
    batch_xs = train_x[randindx, :]
    batch_ys = train_y[randindx, :]
    __, loss = sess.run([optim, L], feed_dict={
        X: batch_xs,
        y: batch_ys
        })
    if _ % log_interval == 0:
        print("Loss:", loss)
accuracy = sess.run(acc, feed_dict={
    X: test_x,
    y: test_y
    })
print("Accuracy:", accuracy)
print("Prediction:", sess.run(_pred, feed_dict={
    X: input_data.reshape((1, ndim)),
    y: np.ones((ndim, 1))
    }))
sess.close()

Loss: 6.08522
Loss: 0.832879
Loss: 0.66764
Loss: 0.687473
Loss: 0.637096
Loss: 0.627896
Loss: 0.60689
Loss: 0.702141
Loss: 0.671151
Loss: 0.636966
Loss: 0.539395
Loss: 0.588333
Loss: 0.580894
Loss: 0.609481
Loss: 0.632672
Loss: 0.616814
Loss: 0.636228
Loss: 0.62248
Loss: 0.663693
Loss: 0.665373
Loss: 0.582697
Loss: 0.633728
Loss: 0.651244
Loss: 0.618371
Loss: 0.653285
Loss: 0.620975
Loss: 0.619191
Loss: 0.625438
Loss: 0.597447
Loss: 0.714343
Loss: 0.590965
Loss: 0.601308
Loss: 0.690823
Loss: 0.605712
Loss: 0.632658
Loss: 0.556844
Loss: 0.65082
Loss: 0.611267
Loss: 0.733543
Loss: 0.630462
Loss: 0.591031
Loss: 0.662758
Loss: 0.604838
Loss: 0.612028
Loss: 0.668958
Loss: 0.614918
Loss: 0.643426
Loss: 0.623833
Loss: 0.599879
Loss: 0.633377
Loss: 0.620719
Loss: 0.603391
Loss: 0.674292
Loss: 0.564408
Loss: 0.616056
Loss: 0.669452
Accuracy: 0.7625
Prediction: [[ 0.24410546]]
